In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from os import path
from run_snndensity import read_dataset,read_snnArray,dbscan
import sys
sys.path.append('../../')
from Evaluation.silhouette_coefficient import silhoutte_result
snn_nparray = read_snnArray('../snn_computation/snn_distance_metric.csv')


In [2]:
#DB SCAN

def demo(input_minpts,input_eps):
    data_labeled = dbscan(MinPts=input_minpts,Eps=input_eps,snn_nparray=snn_nparray)
    labels = data_labeled.labels_
    # print(labels)

# Remove outliers
    count = 0
    for item in labels:
        if item>-1:
            count = count+1
    # print("Number of true node ", count)  # Number of node after remove outliers

    new_distance_metric = np.zeros((count,count),dtype=float)
    label_dictionary_before = {}
    
    start = 0 
    for item in labels:
        label_dictionary_before[start]=item 
        start = start+1
    # print(label_dictionary_before)
    dictionary_no_outliers = {i:label_dictionary_before[i] for i in label_dictionary_before if label_dictionary_before[i]!=-1}
    # print(dictionary_no_outliers)

    k = dictionary_no_outliers.keys()

    # Create a new name for each node after remove outliers
    change_name_list={}
    start = 0
    for item in k:
        change_name_list[start] = item
        start = start+1

    #Add value to new snn_array, to use for calculate  Silhoutte - after remove outliers
    i=0
    j=0
    for i in range(0,count):
        for j in range(i+1,count):
            new_distance_metric[i][j] = snn_nparray[change_name_list[i]][change_name_list[j]]
            new_distance_metric[j][i] = new_distance_metric[i][j]

    # Silhoutte 
    new_label = list(dictionary_no_outliers.values())
    number_of_cluster = len(set(new_label)) 
    if len(set(new_label)) > 1:
        silhoute_removeouliers= silhoutte_result(new_distance_metric,new_label)
    else:
        print("Number of labels needs to > 1. ERROR!!")
        silhoute_removeouliers = -1

    return silhoute_removeouliers,len(new_label),number_of_cluster,label_dictionary_before


In [10]:
list_minpts = [i for i in range (3,20)]
eps = 0.65
print("Silhoutte \t Number of Clusters \t Number of nodes")
for item in list_minpts:
    score,number_of_nodes,num_clusters,label_dictionary = demo(item,eps)
    print(score, num_clusters,number_of_nodes)

Silhoutte 	 Number of Clusters 	 Number of nodes
0.5196751942481924 19 129
0.5100750005371928 12 108
0.517969882717201 7 88
0.508209032278478 6 83
0.5390059075669844 6 78
0.506701325683917 5 71
0.506701325683917 5 71
0.5003226645849598 4 61
0.5040896358543417 3 51
0.5040896358543417 3 51
0.5277210884353741 2 35
Number of labels needs to > 1. ERROR!!
-1 1 22
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20
Number of labels needs to > 1. ERROR!!
-1 1 20


In [11]:
list_eps=np.arange(0.05, 1, 0.05).tolist()
for item in list_eps:
    score,number_of_nodes,num_clusters,label_dictionary = demo(2,item)
    print(score, num_clusters,number_of_nodes)





0.7932184353004214 20 72
0.756106650555935 24 93
0.7691852630026038 27 103
0.7200716907872788 30 112
0.7550900773252771 32 120
0.7189566755760081 36 135
0.691094855222317 38 144
0.6664600668535048 42 157
0.6568614967365756 46 167
0.6394588041227534 52 181
0.5918509261529006 55 196
0.5742705145674987 61 210
0.5437769251459266 68 227
0.5216897677942888 78 248
0.46860608827862343 96 293
0.34814728315897353 145 426
0.20145674999047672 272 838
0.09639488660281718 576 2014
0.022828595481381782 509 4704


In [5]:
# label_dictionary_before = {}
# start = 0 
# for item in labels:
#     label_dictionary_before[start]=item 
#     start = start+1
# print(label_dictionary_before)

In [6]:
# dictionary_no_outliers = {i:label_dictionary_before[i] for i in label_dictionary_before if label_dictionary_before[i]!=-1}
# print(dictionary_no_outliers)

In [7]:
# k = dictionary_no_outliers.keys()
# print(k)
# change_name_list={}
# start = 0
# for item in k:
#     change_name_list[start] = item
#     start = start+1
# print(change_name_list)